In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd

%matplotlib notebook
import matplotlib.pyplot as plt

from pandas_profiling import ProfileReport
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
import lightgbm
import xgboost as xgb

In [ ]:
train = pd.read_csv("/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_train.csv")

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
profile = ProfileReport(train)

In [ ]:
profile

In [ ]:
def treat_data(df):
    
    #df = df.loc[:, df.columns[~ df.columns.str.contains("_id")]]
    df.dropna(axis=0, inplace=True)

    df.reset_index(inplace=True)
    df.drop("index", axis=1, inplace=True)

    mapping = df.city.value_counts()
    df.city = df.city.map(mapping)
    df = pd.get_dummies(df, columns=["gender"], drop_first=True)

    rev_exp_ord_encoder = ce.OrdinalEncoder(
    mapping=[
        {
            "col":"relevent_experience",
            "mapping":{
                "No relevent experience" : 0,
                "Has relevent experience" : 1
            }
        }
    ]
    )
    df = rev_exp_ord_encoder.fit_transform(df)



    univ_ord_encoder = ce.OrdinalEncoder(
    mapping=[
        {
            "col":"enrolled_university",
            "mapping":{
                "no_enrollment" : 0,
                "Part time course" : 1,
                "Full time course" : 2
            }
        }
    ]
    )

    df = univ_ord_encoder.fit_transform(df)

    ed_ord_encoder = ce.OrdinalEncoder(
    mapping=[
        {
            "col":"education_level",
            "mapping":{
                "Graduate" : 1,
                "Masters" : 2,
                "Phd" : 3
            }
        }
    ]
    )
    df = ed_ord_encoder.fit_transform(df)


    df = pd.get_dummies(df, columns=["major_discipline"], drop_first=True)


    ex_ord_encoder = ce.OrdinalEncoder(
    mapping=[
        {
            "col":"experience",
            "mapping":{
                "<1" : 0,
                "1" : 1,
                "2" : 2,
                "4" : 4,
                "5" : 5,
                "6" : 6,
                "7" : 7,
                "8" : 8,
                "9" : 9,
                "10" : 10,
                "11" : 11,
                "12" : 12,
                "13" : 13,
                "14" : 14,
                "15" : 15,
                "16" : 16,
                "17" : 17,
                "18" : 18,
                "19" : 19,
                "20" : 20,
                ">20": 21
            }
        }
    ]
    )
    df = ex_ord_encoder.fit_transform(df)


    comp_size_ord_encoder = ce.OrdinalEncoder(
    mapping=[
        {
            "col":"company_size",
            "mapping":{
                "<10" : 0,
                "10/49" : 1,
                "50-99" : 2,
                "100-500" : 3,
                "500-999" : 4,
                "1000-4999" : 5,
                "5000-9999" : 6,
                "10000+" : 7,
            }
        }
    ]
    )

    df = comp_size_ord_encoder.fit_transform(df)

    df = pd.get_dummies(df, columns = ["company_type"], drop_first=True)


    last_n_j_ord_encoder = ce.OrdinalEncoder(
    mapping=[
        {
            "col":"last_new_job",
            "mapping":{
                "never" : 0,
                "1" : 1,
                "2" : 2,
                "3" : 3,
                "4" : 4,
                ">4" : 5
            }
        }
    ]
    )

    df = last_n_j_ord_encoder.fit_transform(df)

    df["experience"] = df["experience"].astype("int32")

    #df["target"] = df["target"].astype("uint8")

    sca = StandardScaler()

    df.loc[:, ["city_development_index", "training_hours"]] = sca.fit_transform(df.loc[:, ["city_development_index", "training_hours"]])
    
    return df

In [ ]:
def run_kfold(X, y, model, folds=10):
    skf = StratifiedKFold(n_splits=folds)
    skf.get_n_splits(X, y)
    roc_auc_avg = 0
    k = 0
    for train_index, test_index in skf.split(X, y):
        k+=1
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
        y_train, y_test = y[train_index], y[test_index]


        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        
        roc_auc_avg += roc_auc_score(y_pred, y_test)

      
        print("\nfold " + str(k) + " " + str(roc_auc_score(y_pred, y_test)))
        print('Accuracy: %f, \nRecall: %f \nPrecision: %f'
          % (accuracy_score(y_test, y_pred), recall_score(y_test, y_pred), precision_score(y_test, y_pred)))



    print("avg roc_auc score is " + str(roc_auc_avg/k))
    

In [ ]:
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn import linear_model
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, precision_score

In [ ]:
train = treat_data(train)

In [ ]:
X = train.drop(["target", "enrollee_id"], axis=1)
y = train.target

In [ ]:
X.dtypes

In [ ]:
lg = linear_model.LogisticRegression(max_iter=3000)
run_kfold(X, y, lg, folds=10)

In [ ]:
rfc = RandomForestClassifier(random_state=0)
run_kfold(X, y, rfc, folds=10)

# RFE

In [ ]:
rfe = RFE(rfc)
rfe.fit(X, y)

In [ ]:
rfe.ranking_

In [ ]:
rfe.estimator_.n_features_

In [ ]:
rfe.support_

In [ ]:
rfe_cols = X.columns[rfe.support_]

In [ ]:
X_rfe = X.loc[:, rfe_cols]

In [ ]:
from sklearn.utils import resample

In [ ]:
len(train[train.target==0])

In [ ]:
# Separate majority and minority classes
train_majority = train[train.target==0]
train_minority = train[train.target==1]
 
# Upsample minority class
train_minority_upsampled = resample(train_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=7472,    # to match majority class
                                 random_state=0) # reproducible results
 
# Combine majority class with upsampled minority class
train_upsampled = pd.concat([train_majority, train_minority_upsampled])
 
# Display new class counts
train_upsampled.target.value_counts()
# 1    576
# 0    576
# Name: balance, dtype: int64

In [ ]:
train_upsampled.reset_index(inplace=True)
train_upsampled.drop("index", axis=1, inplace=True)
X_up_rfe = train_upsampled[X_rfe.columns]
y_up_rfe = train_upsampled.target

# logistic reg

In [ ]:
lg = linear_model.LogisticRegression(random_state=0, max_iter=3000)
run_kfold(X_up_rfe, y_up_rfe, lg, folds=10)

# random forest

In [ ]:
rfc = RandomForestClassifier(random_state=0)
run_kfold(X_up_rfe, y_up_rfe, rfc, folds=10)

# xgboost

In [ ]:
xgbc = xgb.XGBClassifier()
run_kfold(X_up_rfe, y_up_rfe, xgbc, folds=10)

# light gbm

In [ ]:
lgbm = lightgbm.LGBMClassifier(random_state=0)
run_kfold(X_up_rfe, y_up_rfe, lgbm, folds=10)

# test data

In [ ]:
test = pd.read_csv("/kaggle/input/hr-analytics-job-change-of-data-scientists//aug_test.csv")

In [ ]:
test = treat_data(test)

In [ ]:
test_prob = rfc.predict_proba(test[X_rfe.columns])

In [ ]:
test_prob = [ i[1] for i in test_prob]

In [ ]:
result = pd.DataFrame({
    "enrollee_id" : test["enrollee_id"],
    "targer" : test_prob
})

In [ ]:
result.to_csv("submission.csv", index=False)